# Getting started with Azure ML Data Prep SDK
Copyright (c) Microsoft Corporation. All rights reserved.<br>
Licensed under the MIT License.
#### Note: Some features in this Notebook will _not_ work with the Private Preview version of the SDK; it assumes the Public Preview version.

Wonder how you can make the most of the Azure ML Data Prep SDK? In this "Getting Started" guide, we'll showcase a few highlights that make this SDK shine for big datasets where `pandas` and `dplyr` can fall short. Using the [Ford GoBike dataset](https://www.fordgobike.com/system-data) as an example, we'll cover how to build Dataflows that allow you to:

* [Read in data](#Read-in-data)
* [Get a profile of your data](#Get-data-profile)
* [Apply smart transforms by Microsoft Research](#Derive-by-example)
* [Filter quickly](#Filter-our-data)
* [Apply common data science transforms](#Transform-our-data)
* [Easily handle errors and assertions](#Assert-on-invalid-data)
* [Prepare your dataset for export and machine learning](#Export-for-machine-learning)

In [1]:
from IPython.display import display
from os import path
from tempfile import mkdtemp

import pandas as pd
import azureml.dataprep as dprep

## Read in data

Azure ML Data Prep supports many different file reading formats (i.e. CSV, Excel, Parquet), and also offers the ability to infer column types automatically. 

In [2]:
gobike = dprep\
    .read_csv(
        path='https://dprepdata.blob.core.windows.net/demo/ford_gobike/2017-fordgobike-tripdata.csv',
        inference_arguments=dprep.InferenceArguments.current_culture()
    )
gobike.head(5)

,duration_sec,start_time,end_time,start_station_id,start_station_name,start_station_latitude,start_station_longitude,end_station_id,end_station_name,end_station_latitude,end_station_longitude,bike_id,user_type,member_birth_year,member_gender
0,80110.0,2017-12-31 16:57:39.654,2018-01-01 15:12:50.245,74.0,Laguna St at Hayes St,37.776435,-122.426244,43.0,San Francisco Public Library (Grove St at Hyde...,37.778768,-122.415929,96.0,Customer,1987.0,Male
1,78800.0,2017-12-31 15:56:34.842,2018-01-01 13:49:55.617,284.0,Yerba Buena Center for the Arts (Howard St at ...,37.784872,-122.400876,96.0,Dolores St at 15th St,37.766210,-122.426614,88.0,Customer,1965.0,Female
2,45768.0,2017-12-31 22:45:48.411,2018-01-01 11:28:36.883,245.0,Downtown Berkeley BART,37.870348,-122.267764,245.0,Downtown Berkeley BART,37.870348,-122.267764,1094.0,Customer,NaN,
3,62172.0,2017-12-31 17:31:10.636,2018-01-01 10:47:23.531,60.0,8th St at Ringold St,37.774520,-122.409449,5.0,Powell St BART Station (Market St at 5th St),37.783899,-122.408445,2831.0,Customer,NaN,
4,43603.0,2017-12-31 14:23:14.001,2018-01-01 02:29:57.571,239.0,Bancroft Way at Telegraph Ave,37.868813,-122.258764,247.0,Fulton St at Bancroft Way,37.867789,-122.265896,3167.0,Subscriber,1997.0,Female


In order to iterate more quickly, we can take a sample of our data. Later, we can then apply the same transformations to the entire dataset.

In [3]:
sampled_gobike = gobike.take_sample(probability=0.1, seed=5)

## Get data profile

Let's understand what our data looks like. Azure ML Data Prep facilitates this process by offering data profiles that help us glimpse into column types and column summary statistics.

In [4]:
gobike.get_profile()

,Type,Min,Max,Count,Missing Count,Not Missing Count,Percent missing,Error Count,Empty count,0.1% Quantile,1% Quantile,5% Quantile,25% Quantile,50% Quantile,75% Quantile,95% Quantile,99% Quantile,99.9% Quantile,Standard Deviation,Mean
member_gender,FieldType.STRING,,Other,519700.0,0.0,519700.0,0.000000,0.0,66462.0,,,,,,,,,,,
end_station_latitude,FieldType.DECIMAL,37.3173,37.8802,519700.0,0.0,519700.0,0.000000,0.0,0.0,37.3258,37.7615,37.7612,37.7742,37.7833,37.7956,37.8315,37.8691,37.8737,0.0862238,37.7718
end_time,FieldType.DATE,2017-06-28 09:52:55.338000+00:00,2018-01-01 15:12:50.245000+00:00,519700.0,0.0,519700.0,0.000000,0.0,0.0,,,,,,,,,,,
end_station_id,FieldType.DECIMAL,3,340,519700.0,0.0,519700.0,0.000000,0.0,0.0,3,10.2844,9.34283,22.7024,65.2261,134.22,284.099,323.238,326.494,84.9695,92.184
user_type,FieldType.STRING,Customer,Subscriber,519700.0,0.0,519700.0,0.000000,0.0,0.0,,,,,,,,,,,
duration_sec,FieldType.DECIMAL,61,86369,519700.0,0.0,519700.0,0.000000,0.0,0.0,72.5047,253.621,246.155,381.842,595.984,938.574,2247.33,11651.3,59935.9,3444.15,1099.01
end_station_name,FieldType.STRING,10th Ave at E 15th St,Yerba Buena Center for the Arts (Howard St at ...,519700.0,0.0,519700.0,0.000000,0.0,0.0,,,,,,,,,,,
start_station_longitude,FieldType.DECIMAL,-122.444,-121.874,519700.0,0.0,519700.0,0.000000,0.0,0.0,-122.444,-122.423,-122.426,-122.412,-122.399,-122.391,-122.251,-121.886,-121.877,0.105573,-122.364
start_station_id,FieldType.DECIMAL,3,340,519700.0,0.0,519700.0,0.000000,0.0,0.0,3,12.3902,10.4053,23.8481,67.1843,139.424,290.611,323.595,325.782,86.0831,95.0342
start_station_latitude,FieldType.DECIMAL,37.3173,37.8802,519700.0,0.0,519700.0,0.000000,0.0,0.0,37.326,37.761,37.7604,37.7736,37.7832,37.7953,37.8324,37.8693,37.8738,0.086305,37.7717


In [5]:
sampled_gobike.get_profile()

,Type,Min,Max,Count,Missing Count,Not Missing Count,Percent missing,Error Count,Empty count,0.1% Quantile,1% Quantile,5% Quantile,25% Quantile,50% Quantile,75% Quantile,95% Quantile,99% Quantile,99.9% Quantile,Standard Deviation,Mean
member_gender,FieldType.STRING,,Other,51853.0,0.0,51853.0,0.000000,0.0,6571.0,,,,,,,,,,,
end_station_latitude,FieldType.DECIMAL,37.3184,37.8802,51853.0,0.0,51853.0,0.000000,0.0,0.0,37.3262,37.7614,37.7611,37.7745,37.7836,37.7956,37.8306,37.869,37.8738,0.0861915,37.7719
end_time,FieldType.DATE,2017-06-28 11:01:39.557000+00:00,2018-01-01 15:12:50.245000+00:00,51853.0,0.0,51853.0,0.000000,0.0,0.0,,,,,,,,,,,
end_station_id,FieldType.DECIMAL,3,338,51853.0,0.0,51853.0,0.000000,0.0,0.0,3,9.82,8.74048,22.3474,65.6089,135.081,283.542,323.152,326.322,85.0916,91.9201
user_type,FieldType.STRING,Customer,Subscriber,51853.0,0.0,51853.0,0.000000,0.0,0.0,,,,,,,,,,,
duration_sec,FieldType.DECIMAL,61,85864,51853.0,0.0,51853.0,0.000000,0.0,0.0,73.8534,254.265,248.086,381.017,596.082,936.399,2229.13,11921.8,61331.9,3527.18,1102.23
end_station_name,FieldType.STRING,10th Ave at E 15th St,Yerba Buena Center for the Arts (Howard St at ...,51853.0,0.0,51853.0,0.000000,0.0,0.0,,,,,,,,,,,
start_station_longitude,FieldType.DECIMAL,-122.444,-121.874,51853.0,0.0,51853.0,0.000000,0.0,0.0,-122.444,-122.423,-122.423,-122.412,-122.399,-122.391,-122.251,-121.886,-121.877,0.105593,-122.364
start_station_id,FieldType.DECIMAL,3,340,51853.0,0.0,51853.0,0.000000,0.0,0.0,3,11.8549,10.9241,23.823,66.8145,139.679,292.979,323.616,325.724,86.0923,94.8785
start_station_latitude,FieldType.DECIMAL,37.3173,37.8802,51853.0,0.0,51853.0,0.000000,0.0,0.0,37.3261,37.761,37.7608,37.7736,37.7833,37.7954,37.8324,37.8692,37.8737,0.0862637,37.7717


It appears that we have quite a few missing values in `member_birth_year`. We also immediately see that we have some empty strings in our `member_gender` column. With the data profiler, we can quickly do a sanity check on our dataset and see where we might need to start data cleaning.

## Derive by example

Azure ML Data Prep comes with additional "smart" transforms created by Microsoft Research. Here, we'll look at how you can derive a new column by providing examples of input-output pairs. Rather than explicitly using regular expressions to extract dates or hours from datetimes, we can provide examples for Azure ML Data Prep to learn what the pattern is. In fact, these smart transformations can also handle more complex derivations like inferring the day of the week from datetimes.

In [6]:
sgb_derived = sampled_gobike\
    .to_string(
        columns=['start_time', 'end_time']
    )\
    .derive_column_by_example(
        source_columns='start_time',
        new_column_name='date',
        example_data=[('2017-12-31 16:57:39.6540', '2017-12-31'), ('2017-12-31 16:57:39', '2017-12-31')]
    )\
    .derive_column_by_example(
        source_columns='start_time',
        new_column_name='hour',
        example_data=[('2017-12-31 16:57:39.6540', '16')]
    )\
    .derive_column_by_example(
        source_columns='start_time',
        new_column_name='wday',
        example_data=[('2017-12-31 16:57:39.6540', 'Sunday')]
    )

## Filter our data

Let's verify that our derivations are correct by doing a bit of spot-checking.

In [7]:
sgb_derived.filter(dprep.col('wday') != 'Sunday').head(5)

,duration_sec,start_time,wday,hour,date,end_time,start_station_id,start_station_name,start_station_latitude,start_station_longitude,end_station_id,end_station_name,end_station_latitude,end_station_longitude,bike_id,user_type,member_birth_year,member_gender
0,3456.0,2017-12-30 23:46:13.358000,Saturday,23,2017-12-30,2017-12-31 00:43:49.469000,75.0,Market St at Franklin St,37.773793,-122.421239,75.0,Market St at Franklin St,37.773793,-122.421239,1642.0,Subscriber,1972.0,Male
1,204.0,2017-12-30 23:31:38.904000,Saturday,23,2017-12-30,2017-12-30 23:35:03.121000,84.0,Duboce Park,37.769200,-122.433812,107.0,17th St at Dolores St,37.763015,-122.426497,2201.0,Subscriber,1965.0,Male
2,743.0,2017-12-30 22:35:13.114000,Saturday,22,2017-12-30,2017-12-30 22:47:36.356000,285.0,Webster St at O'Farrell St,37.783521,-122.431158,97.0,14th St at Mission St,37.768265,-122.420110,1628.0,Subscriber,1993.0,Male
3,328.0,2017-12-30 22:19:28.760000,Saturday,22,2017-12-30,2017-12-30 22:24:57.489000,5.0,Powell St BART Station (Market St at 5th St),37.783899,-122.408445,64.0,5th St at Brannan St,37.776754,-122.399018,2806.0,Subscriber,1986.0,Male
4,260.0,2017-12-30 21:22:40.116000,Saturday,21,2017-12-30,2017-12-30 21:27:00.885000,277.0,Morrison Ave at Julian St,37.333658,-121.908586,278.0,The Alameda at Bush St,37.331932,-121.904888,465.0,Subscriber,1991.0,Male


We can also filter on other column types; let's take a peek at rides that lasted over 5 hours.

In [8]:
sgb_derived.filter(dprep.col('duration_sec') > (60 * 60 * 5)).head(5)

,duration_sec,start_time,wday,hour,date,end_time,start_station_id,start_station_name,start_station_latitude,start_station_longitude,end_station_id,end_station_name,end_station_latitude,end_station_longitude,bike_id,user_type,member_birth_year,member_gender
0,80110.0,2017-12-31 16:57:39.654000,Sunday,16,2017-12-31,2018-01-01 15:12:50.245000,74.0,Laguna St at Hayes St,37.776435,-122.426244,43.0,San Francisco Public Library (Grove St at Hyde...,37.778768,-122.415929,96.0,Customer,1987,Male
1,22587.0,2017-12-31 13:51:04.538000,Sunday,13,2017-12-31,2017-12-31 20:07:32.139000,307.0,SAP Center,37.332692,-121.900084,307.0,SAP Center,37.332692,-121.900084,1443.0,Customer,None,
2,18067.0,2017-12-30 04:20:13.938000,Saturday,04,2017-12-30,2017-12-30 09:21:21.628000,70.0,Central Ave at Fell St,37.773311,-122.444293,43.0,San Francisco Public Library (Grove St at Hyde...,37.778768,-122.415929,1928.0,Customer,None,
3,54550.0,2017-12-29 10:02:38.086000,Friday,10,2017-12-29,2017-12-30 01:11:48.539000,21.0,Montgomery St BART Station (Market St at 2nd St),37.789625,-122.400811,84.0,Duboce Park,37.769200,-122.433812,209.0,Customer,None,
4,63627.0,2017-12-27 19:12:42.794000,Wednesday,19,2017-12-27,2017-12-28 12:53:10.649000,249.0,Russell St at College Ave,37.858473,-122.253253,244.0,Shattuck Ave at Hearst Ave,37.873792,-122.268618,1804.0,Customer,1988,Male


## Transform our data

In addition to "smart" transformations, Azure ML Data Prep also supports many common data science transforms familiar to other industry-standard data science libraries. Here, we'll explore the ability to `summarize` and `replace`. We'll also get to use `join` when we handle assertions.

#### Summarize


In [9]:
sgb_summary = sgb_derived\
    .summarize(
        summary_columns=[
            dprep\
                .SummaryColumnsValue(
                    column_id='duration_sec', 
                    summary_column_name='duration_sec_mean', 
                    summary_function=dprep.SummaryFunction.MEAN
                )
        ],
        group_by_columns=['date']
    )
sgb_summary.head(5)

,date,duration_sec_mean
0,2017-12-31,1982.801418
1,2017-12-30,1203.766423
2,2017-12-29,1287.324841
3,2017-12-28,835.146465
4,2017-12-27,1658.735955


Azure Data Prep also makes it easy to append this output of `summarize` to the original table based on the grouping variable. 

In [10]:
sgb_appended = sgb_derived\
    .summarize(
        summary_columns=[
            dprep\
                .SummaryColumnsValue(
                    column_id='duration_sec', 
                    summary_column_name='duration_sec_mean', 
                    summary_function=dprep.SummaryFunction.MEAN
                )
        ],
        group_by_columns=['date'],
        join_back=True
    )
sgb_appended.head(5)

,duration_sec,start_time,wday,hour,date,end_time,start_station_id,start_station_name,start_station_latitude,start_station_longitude,end_station_id,end_station_name,end_station_latitude,end_station_longitude,bike_id,user_type,member_birth_year,member_gender,duration_sec_mean
0,80110.0,2017-12-31 16:57:39.654000,Sunday,16,2017-12-31,2018-01-01 15:12:50.245000,74.0,Laguna St at Hayes St,37.776435,-122.426244,43.0,San Francisco Public Library (Grove St at Hyde...,37.778768,-122.415929,96.0,Customer,1987,Male,1982.801418
1,3292.0,2017-12-31 23:46:32.403000,Sunday,23,2017-12-31,2018-01-01 00:41:24.605000,284.0,Yerba Buena Center for the Arts (Howard St at ...,37.784872,-122.400876,22.0,Howard St at Beale St,37.789756,-122.394643,3058.0,Customer,None,,1982.801418
2,1397.0,2017-12-31 23:55:09.686000,Sunday,23,2017-12-31,2018-01-01 00:18:26.721000,78.0,Folsom St at 9th St,37.773717,-122.411647,15.0,San Francisco Ferry Building (Harry Bridges Pl...,37.795392,-122.394203,1667.0,Customer,None,,1982.801418
3,422.0,2017-12-31 23:54:25.337000,Sunday,23,2017-12-31,2018-01-01 00:01:27.354000,139.0,Garfield Square (25th St at Harrison St),37.751017,-122.411901,99.0,Folsom St at 15th St,37.767037,-122.415442,2415.0,Subscriber,1985,Male,1982.801418
4,1130.0,2017-12-31 23:36:16.069000,Sunday,23,2017-12-31,2017-12-31 23:55:06.096000,66.0,3rd St at Townsend St,37.778742,-122.392741,23.0,The Embarcadero at Steuart St,37.791464,-122.391034,2721.0,Customer,None,,1982.801418


#### Replace

Recall that our `member_gender` column had empty strings that stood in place of `None`. Let's use our `replace` function to properly recode them as `None`s.

In [11]:
sgb_replaced = sampled_gobike.replace_na(columns=['member_gender'])
sgb_replaced.head(5)

,duration_sec,start_time,end_time,start_station_id,start_station_name,start_station_latitude,start_station_longitude,end_station_id,end_station_name,end_station_latitude,end_station_longitude,bike_id,user_type,member_birth_year,member_gender
0,80110.0,2017-12-31 16:57:39.654,2018-01-01 15:12:50.245,74.0,Laguna St at Hayes St,37.776435,-122.426244,43.0,San Francisco Public Library (Grove St at Hyde...,37.778768,-122.415929,96.0,Customer,1987,Male
1,3292.0,2017-12-31 23:46:32.403,2018-01-01 00:41:24.605,284.0,Yerba Buena Center for the Arts (Howard St at ...,37.784872,-122.400876,22.0,Howard St at Beale St,37.789756,-122.394643,3058.0,Customer,None,None
2,1397.0,2017-12-31 23:55:09.686,2018-01-01 00:18:26.721,78.0,Folsom St at 9th St,37.773717,-122.411647,15.0,San Francisco Ferry Building (Harry Bridges Pl...,37.795392,-122.394203,1667.0,Customer,None,None
3,422.0,2017-12-31 23:54:25.337,2018-01-01 00:01:27.354,139.0,Garfield Square (25th St at Harrison St),37.751017,-122.411901,99.0,Folsom St at 15th St,37.767037,-122.415442,2415.0,Subscriber,1985,Male
4,1130.0,2017-12-31 23:36:16.069,2017-12-31 23:55:06.096,66.0,3rd St at Townsend St,37.778742,-122.392741,23.0,The Embarcadero at Steuart St,37.791464,-122.391034,2721.0,Customer,None,None


## Assert on invalid data 

Azure ML Data Prep helps prevent broken pipelines and safeguard against bad data by supporting assertions. In our case, we'll create assertions to handle potentially erroneous `member_birth_year` values. The oldest person on record is no more than 130 years old, so birth year listed as before 1900 is wrong. Though our `sampled_gobike` dataset doesn't have any issues, we would fail on the full `gobike` dataset if we made that assumption. However, Azure ML Data Prep allows us to handle these gracefully with assertions.

In [12]:
gb_asserted = gobike\
    .assert_value(
        columns='member_birth_year', 
        expression=dprep.f_or(dprep.value.is_null(), dprep.value >= 1900),
        error_code='InvalidDate'
    )
gb_asserted.get_profile()

,Type,Min,Max,Count,Missing Count,Not Missing Count,Percent missing,Error Count,Empty count,0.1% Quantile,1% Quantile,5% Quantile,25% Quantile,50% Quantile,75% Quantile,95% Quantile,99% Quantile,99.9% Quantile,Standard Deviation,Mean
member_gender,FieldType.STRING,,Other,519700.0,0.0,519700.0,0.000000,0.0,66462.0,,,,,,,,,,,
end_station_latitude,FieldType.DECIMAL,37.3173,37.8802,519700.0,0.0,519700.0,0.000000,0.0,0.0,37.3258,37.7615,37.7612,37.7742,37.7833,37.7956,37.8315,37.8691,37.8737,0.0862238,37.7718
end_time,FieldType.DATE,2017-06-28 09:52:55.338000+00:00,2018-01-01 15:12:50.245000+00:00,519700.0,0.0,519700.0,0.000000,0.0,0.0,,,,,,,,,,,
end_station_id,FieldType.DECIMAL,3,340,519700.0,0.0,519700.0,0.000000,0.0,0.0,3,10.2844,9.34283,22.7024,65.2261,134.22,284.099,323.238,326.494,84.9695,92.184
user_type,FieldType.STRING,Customer,Subscriber,519700.0,0.0,519700.0,0.000000,0.0,0.0,,,,,,,,,,,
duration_sec,FieldType.DECIMAL,61,86369,519700.0,0.0,519700.0,0.000000,0.0,0.0,72.5047,253.621,246.155,381.842,595.984,938.574,2247.33,11651.3,59935.9,3444.15,1099.01
end_station_name,FieldType.STRING,10th Ave at E 15th St,Yerba Buena Center for the Arts (Howard St at ...,519700.0,0.0,519700.0,0.000000,0.0,0.0,,,,,,,,,,,
start_station_longitude,FieldType.DECIMAL,-122.444,-121.874,519700.0,0.0,519700.0,0.000000,0.0,0.0,-122.444,-122.423,-122.426,-122.412,-122.399,-122.391,-122.251,-121.886,-121.877,0.105573,-122.364
start_station_id,FieldType.DECIMAL,3,340,519700.0,0.0,519700.0,0.000000,0.0,0.0,3,12.3902,10.4053,23.8481,67.1843,139.424,290.611,323.595,325.782,86.0831,95.0342
start_station_latitude,FieldType.DECIMAL,37.3173,37.8802,519700.0,0.0,519700.0,0.000000,0.0,0.0,37.326,37.761,37.7604,37.7736,37.7832,37.7953,37.8324,37.8693,37.8738,0.086305,37.7717


Now, we can filter to see what caused the 2 errors above:

In [13]:
gb_errors = gb_asserted.filter(dprep.col('member_birth_year').is_error())
gb_errors.head(5)

,duration_sec,start_time,end_time,start_station_id,start_station_name,start_station_latitude,start_station_longitude,end_station_id,end_station_name,end_station_latitude,end_station_longitude,bike_id,user_type,member_birth_year,member_gender
0,2546.0,2017-08-19 17:47:32.110,2017-08-19 18:29:58.825,197.0,El Embarcadero at Grand Ave,37.808848,-122.24968,172.0,College Ave at Taft Ave,37.841800,-122.251535,1448.0,Customer,"azureml.dataprep.native.DataPrepError(""'Invali...",Male
1,1767.0,2017-08-19 13:20:02.170,2017-08-19 13:49:29.735,235.0,Union St at 10th St,37.807239,-122.28937,197.0,El Embarcadero at Grand Ave,37.808848,-122.249680,91.0,Customer,"azureml.dataprep.native.DataPrepError(""'Invali...",Male


#### Join
But what were the original values? Let's use `join` to figure out what the values were that caused our assert to throw an error. 

In [14]:
gb_errors.join(
    left_dataflow=gb_errors,
    right_dataflow=gobike,
    join_key_pairs=[
        ('duration_sec', 'duration_sec'),
        ('start_station_id', 'start_station_id'),
        ('bike_id', 'bike_id')
    ]
).head(5)

,l_duration_sec,l_start_time,l_end_time,l_start_station_id,l_start_station_name,l_start_station_latitude,l_start_station_longitude,l_end_station_id,l_end_station_name,l_end_station_latitude,...,r_start_station_latitude,r_start_station_longitude,r_end_station_id,r_end_station_name,r_end_station_latitude,r_end_station_longitude,r_bike_id,r_user_type,r_member_birth_year,r_member_gender
0,2546.0,2017-08-19 17:47:32.110,2017-08-19 18:29:58.825,197.0,El Embarcadero at Grand Ave,37.808848,-122.24968,172.0,College Ave at Taft Ave,37.841800,...,37.808848,-122.24968,172.0,College Ave at Taft Ave,37.841800,-122.251535,1448.0,Customer,1886.0,Male
1,1767.0,2017-08-19 13:20:02.170,2017-08-19 13:49:29.735,235.0,Union St at 10th St,37.807239,-122.28937,197.0,El Embarcadero at Grand Ave,37.808848,...,37.807239,-122.28937,197.0,El Embarcadero at Grand Ave,37.808848,-122.249680,91.0,Customer,1886.0,Male


If we look at `r_member_birth_year`, we see that these people were listed as being born in 1886. That's impossible! Now that we've identified outliers and anomalies, we can appropriately clean our data however we like.

## Export for machine learning

One of the beautiful features of Azure ML Data Prep is that you only need to write your code once and choose whether to scale up or out; it takes care of figuring out how. To do so, you can export the `.dprep` file you've written tested on a smaller dataset, then run it with your larger dataset. Here, we show how you can export your new package. For a more detailed example on how to execute it on Spark, check out our [New York Taxicab scenario](https://github.com/Microsoft/PendletonDocs/blob/master/Scenarios/NYTaxiCab/01.new_york_taxi.ipynb).

In [15]:
gobike = gobike.set_name(name="gobike")
package_path = path.join(mkdtemp(), "gobike.dprep")

print("Saving package to: {}".format(package_path))
package = dprep.Package(arg=gobike)
package.save(file_path=package_path)

Saving package to: /tmp/tmps2f57l94/gobike.dprep


Package
  name: None
  path: /tmp/tmps2f57l94/gobike.dprep
  dataflows: [
    Dataflow {
      name: gobike
      steps: 4
    },
  ]

## Want more information?

Congratulations on finishing your introduction to the Azure ML Data Prep SDK! If you'd like more detailed tutorials on how to construct machine learning datasets or dive deeper into all of its functionality, you can find more information in our detailed notebooks [here](https://github.com/Microsoft/PendletonDocs). There, we cover topics including how to:

* Cache your Dataflow to speed up your iterations
* Add your custom Python transforms
* Impute missing values
* Sample your data
* Reference and link between Dataflows
* Apply your Dataflow to a new, larger data source